# Exercice 1 : librairie time

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Code qui permet de fournir le jour de la semaine à partir d'une date donnée au format dd-mm-yyyy (mois-jour-année).
"""

import time


def find_weekday(str_date):
    """
    à partir d'une date `str_date` (str) au format dd-mm-yyyy,
    retourne
    - une chaîne de caractères avec le jour de la semaine correspondant
    """
    time_object = time.strptime(str_date, "%d-%m-%Y")
    day_indix = time.strftime("%w", time_object)
    weekdays = ["Dimanche", "Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi"]
    return weekdays[int(day_indix)]


if __name__ == "__main__":
    day = "26-04-2100"
    print("Le {} était un {}".format(day, find_weekday(day)))

Le 26-04-2100 était un Lundi


# Exercice 2 : librairie os

In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Code qui permet de créer des fichiers vides au format  'output-dd-mm-yyyy-XX.txt' dans un répertoire `test`
"""

import time
import os


def make_files(N=25, str_date=time.strftime("%d-%m-%Y")):
    """
    Fonction qui créé N fichiers de la forme 'output-dd-mm-yyyy-XX.txt'
    dans le dossier `test` situé dans le répertoire du script d'exécution (sinon, le dossier est créé)
        où dd-mm-yyyy est la date au format jour-mois-année indiquée par str_date.

    input parameters :
    - N (int) : nombre de fichiers à créer
    - str_date (str) : date à mettre dans les noms de fichier, date du jour par défaut.
    """
    # Création du répertoire test s'il n'existe pas déjà
    if not os.path.exists("./test"):
        os.mkdir("test")

    filenames = []
    for i in range(N):
        # attention à bien ajouter 1 pour avoir un premier numéro égal à 1 et s'arrêter à N
        filename = "test/output-{}-{}.txt".format(str_date, i + 1)
        filenames.append(filenames)
        # On créé un fichier vide
        with open(filename, "w") as f:
            pass

    return filenames


if __name__ == "__main__":
    make_files()
    make_files(str_date="21-03-2023")
    make_files(str_date="10-05-2024")
    make_files(str_date="30-04-2022")

Il faut changer le format des fichiers en : 'output-yyyy-mm-dd-XXXX.txt' où XXXX est maintenant une chaîne de caractères avec des zéros qui servent pour faire du padding.

In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Code qui permet de créer des fichiers vides au format  'output-dd-mm-yyyy-XX.txt' dans un répertoire `test`
"""

import time
import os
import glob


def rename_files(str_date=time.strftime("%d-%m-%Y")):
    """
    Fonction qui renomme N fichiers de la forme 'output-dd-mm-yyyy-XX.txt'
    dans le dossier `test` situé dans le répertoire du script d'exécution (sinon, le dossier est créé)
    au format  'output-yyyy-mm-dd-XXXX.txt' où XXXX est le

    input parameters :
    - str_date (str) : date correspondant aux fichiers à renommer, date du jour par défaut.
    """

    filenames = glob.glob("test/output-{}-*.txt".format(str_date))
    for old_filename in filenames:
        time_obj = time.strptime(str_date, "%d-%m-%Y")
        str_date_new = time.strftime("%Y-%m-%d", time_obj)
        indix = int(old_filename.split("-")[-1].replace(".txt", ""))
        new_filename = "test/output-{}-{:04d}.txt".format(str_date_new, indix)
        os.rename(old_filename, new_filename)
    return filenames


if __name__ == "__main__":
    if os.path.exists("./test"):
        rename_files()
        rename_files(str_date="21-03-2023")
        rename_files(str_date="10-05-2024")
        rename_files(str_date="30-04-2022")
    else:
        print("Le répertoire `test` n'existe pas")

## Exo 2 bis

In [89]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Récupération des noms de fichiers, création des dossiers et tri par date et numéro d'expérience
"""
import time
import datetime
import os
import glob
import shutil


# Programme principal
if __name__ == "__main__":
    filenames = glob.glob("file-exo2-td3/*")
    exps = ["1H", "13C", "HMBC", "HSQC"]
    # Création des dossiers pour chaque type d'expérience
    for exp in exps:
        if not os.path.exists("file-exo2-td3/" + exp):
            os.mkdir("file-exo2-td3/" + exp)
    for file in filenames:
        # On découpe avec les caractères "_"
        # puis on récupère le nom de l'expérience  qui sera le dernier élément de la liste une fois l'extension ".dat" enlevée
        chunksFilename = file.strip(".dat").split("_")
        # Ici on prend la date de modification, mais on pourrait prendre la date de création dans un cas plus concret avec
        # modificationTime = os.path.getctime(file)
        modificationTime = os.path.getmtime(file)
        # On passe ce temps (secondes vs epoch) en objet datetime
        objModificationTime = datetime.datetime.fromtimestamp(modificationTime)
        # On va repasser en date "normalisée" pour avoir le tri alphabétique
        strModificationTime = objModificationTime.strftime("%Y-%m-%d")
        # On vire la date et le nom du dossier
        chunksFilename.pop(0)
        # On prépare le chemin pour la copie des fichiers
        shortFilename = "file-exo2-td3/{}/{}_{}.dat".format(
            chunksFilename[-1], strModificationTime, "_".join(chunksFilename)
        )
        print(shortFilename)
        # On effectue la copie en gardant les méta-données au maximum
        shutil.copy2(file, shortFilename)

In [90]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Script utilisé pour créer l'exercice
"""

import time
import datetime
import os
import pandas as pd
import numpy as np
import filedate
import shutil

# Programme principal
if __name__ == "__main__":
    if not os.path.exists("../file-exo2-td3"):
        os.mkdir("../file-exo2-td3")

    dateFormats = ["%d-%m-%Y", "%m-%d-%Y", "%Y-%m-%d", "%Y-%m-%d", "%y-%m-%d"]
    molecules = pd.read_csv("../molecules-names.csv", delimiter=";")
    prevTime = datetime.date(2020, 3, 17)

    for index, molecule in molecules.iterrows():
        # On créé une date ultérieure à la précédente
        nextTime = prevTime + datetime.timedelta(days=np.random.randint(0, 30))

        molecule["Date"] = index
        # On mélange les colonnes
        test = molecule.sample(frac=1)
        name = ""
        for colNames, col in test.items():
            name += "_" + str(col).replace(" ", "").replace(",", "").replace(
                "'", ""
            )  # .apply("-".join)#.apply(lambda c: c.agg('|'.join, axis=1))
        name = nextTime.strftime(dateFormats[np.random.randint(0, 5)]) + name
        # print(name)
        prevTime = nextTime
        # On créé le fichier avec l'extension, action équivalente à un touch
        for exp in ["1H", "13C", "HMBC", "HSQC"]:
            filepath = "../file-exo2-td3/" + name + "_" + exp + ".dat"
            with open(filepath, "w") as f:
                pass

            # Modification de la dernière date d'accès
            creationTime = datetime.datetime.combine(
                nextTime,
                datetime.time(np.random.randint(0, 24), np.random.randint(0, 60), 0),
            )
            modificationTime = datetime.datetime.combine(
                nextTime + datetime.timedelta(days=np.random.randint(0, 200)),
                datetime.time(np.random.randint(0, 24), np.random.randint(0, 60), 0),
            )
            # os.system("touch {} -t {}".format(filepath,creationTime.strftime('%Y%m%d%H%M'),modificationTime.strftime('%Y%m%d%H%M')))
            os.utime(filepath, (modificationTime.timestamp(), creationTime.timestamp()))

## Exercice 3

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
calcule la position d'équilibre du cerceau en fonction des paramètres d'entrées en unité MKSA

Les paramètres g, r, w peuvent être changés lors de l'appel du script
"""

# Importation des librairies
import math
import os
import argparse

# Definition des fonctions
def posEquilibre(g, r, w):
    """
    calcule la position d'équilibre du cerceau en fonction des paramètres d'entrées en unité MKSA
    - g (float) : gravité en m^2/s
    - r (float) : rayon du cerceau (m)
    - w (float) : vitesse de rotation en rad/s
    retourne
    - thetaEq (float) en radian
    - w0 (float) en rad/s
    """
    wc = math.sqrt(g / r)
    if w <= wc:
        thetaEq = math.pi
        w0 = wc * math.sqrt(1.0 - (w / wc) ** 2)
    else:
        thetaEq = math.acos(-((wc / w) ** 2))
        w0 = w * math.sqrt(1.0 - (wc / w) ** 4)
    return thetaEq, w0


def readingParameters(filename):
    """
    format attendu : parametername value
    avec parametername qui vaut gravity, radius ou omega et value un nombre réel
    """
    r = 1.0
    g = 9.81
    w = 2.0
    argFound = 0
    if os.path.exists(filename):
        with open(filename, "r") as file:
            for line in file:
                if "gravity" in line:
                    g = float(line.split()[1])
                    argFound += 1
                if "radius" in line:
                    r = float(line.split()[1])
                    argFound += 1
                if "omega" in line:
                    w = float(line.split()[1])
                    argFound += 1
    else:
        print(
            "Le fichier d'entrée n'existe pas, les valeurs par défaut seront utilisées"
        )

    if argFound == 3:
        return g, r, w
    else:
        print(
            "Le fichier d'entrée contient moins de paramètres que le minimum nécessaire, les valeurs par défaut seront utilisées pour les valeurs manquantes"
        )
        return g, r, w


# Programme principal
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "-g", "--gravity", default=9.81, type=float, help="gravité en m^2/s"
    )
    parser.add_argument("-r", "--radius", default=1.0, type=float, help="rayon en m")
    parser.add_argument(
        "-w", "--omega", default=2.0, type=float, help="pulsation en rad/s"
    )
    parser.add_argument("-f", "--inputfile", type=str, help="fichier d'input")
    args = parser.parse_args()

    if args.inputfile is not None:
        print(
            "Tous les arguments sont lus à partir du fichier {}".format(args.inputfile)
        )
        g, r, w = readingParameters(args.inputfile)
    else:
        g, r, w = args.gravity, args.radius, args.omega
    thetaEq, w0 = posEquilibre(g, r, w)
    print(
        "Les paramètres du probleme sont : g = {} r = {} w = {} [MKSA]".format(g, r, w)
    )
    print(
        "La position d’équilibre stable est : theta_eq = {} \n\
    La pulsation des petites oscillations autour de cet équilbre est : omega_0 = {} [MKSA]".format(
            thetaEq, w0
        )
    )

Tous les arguments sont lus à partir du fichier /home/mverot/.local/share/jupyter/runtime/kernel-d07309f8-8ab2-4224-98e6-8c8436b62cb7.json
Le fichier d'entrée contient moins de paramètres que le minimum nécessaire, les valeurs par défaut seront utilisées pour les valeurs manquantes
Les paramètres du probleme sont : g = 9.81 r = 1.0 w = 2.0 [MKSA]
La position d’équilibre stable est : theta_eq = 3.141592653589793 
La pulsation des petites oscillations autour de cet équilbre est : omega_0 = 2.41039415863879 [MKSA]
